In [ ]:
import numpy as np
from keras.models import Model, Sequential
from keras.layers import Input, Add, Convolution2D, ZeroPadding2D,Cropping2D, MaxPooling2D, Flatten, Dense, Dropout, Activation, BatchNormalization, ReLU, Softmax, Conv2DTranspose, Concatenate
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing import image
from scipy.io import loadmat

In [ ]:
dataP = loadmat('./kernels/net_P_P_S_F.mat', matlab_compatible=False, struct_as_record=False)
dataG = loadmat('./kernels/net_G_P_S_F.mat', matlab_compatible=False, struct_as_record=False)

In [ ]:
netP = dataP['net_P']
netG = dataG['net_G']

## Parsing Network 

In [ ]:
inputImg = Input(shape=(128,128,3))
inputImg0 = Input(shape=(64,64,3))
zeroPad_1 = ZeroPadding2D((1,1), name= "P_zero_1")(inputImg)
convImg_1 = Convolution2D(16, (3, 3), strides=(1,1), name= "P_conv_1")(zeroPad_1)
batchImg_1 = BatchNormalization(epsilon=1.0000e-04, name= "P_batch_1")(convImg_1)
reluImg_1 = ReLU(name= "P_relu_1")(batchImg_1)
poolImg_1 = MaxPooling2D(pool_size=(2,2), strides=(2,2), name= "P_maxpool_1")(reluImg_1)

zeroPad_2 = ZeroPadding2D((1,1), name= "P_zero_2")(poolImg_1)
convImg_2 = Convolution2D(32, (3, 3), strides=(1,1), name= "P_conv_2")(zeroPad_2)
batchImg_2 = BatchNormalization(epsilon=1.0000e-04, name= "P_batch_2")(convImg_2)
reluImg_2 = ReLU(name= "P_relu_2")(batchImg_2)
poolImg_2 = MaxPooling2D(pool_size=(2,2), strides=(2,2), name= "P_maxpool_2")(reluImg_2)

zeroPad_3 = ZeroPadding2D((2,2), name= "P_zero_3")(poolImg_2)
convImg_3 = Convolution2D(64, (5, 5), strides=(1,1), name= "P_conv_3")(zeroPad_3)
batchImg_3 = BatchNormalization(epsilon=1.0000e-04, name= "P_batch_3")(convImg_3)
reluImg_3 = ReLU(name= "P_relu_3")(batchImg_3)
poolImg_3 = MaxPooling2D(pool_size=(2,2), strides=(2,2), name= "P_maxpool_3")(reluImg_3)

zeroPad_4 = ZeroPadding2D((1,1), name= "P_zero_4")(poolImg_3)
convImg_4 = Convolution2D(128, (3, 3), strides=(1,1), name= "P_conv_4")(zeroPad_4)
batchImg_4 = BatchNormalization(epsilon=1.0000e-04, name= "P_batch_4")(convImg_4)
reluImg_4 = ReLU(name= "P_relu_4")(batchImg_4)
poolImg_4 = MaxPooling2D(pool_size=(2,2), strides=(2,2), name= "P_maxpool_4")(reluImg_4)

zeroPad_5 = ZeroPadding2D((1,1), name= "P_zero_5")(poolImg_4)
convImg_5  = Convolution2D(256, (3, 3), strides=(1,1), name= "P_conv_5")(zeroPad_5)
batchImg_5 = BatchNormalization(epsilon=1.0000e-04, name= "P_batch_5")(convImg_5)
reluImg_5  = ReLU(name= "P_relu_5")(batchImg_5)
poolImg_5 = MaxPooling2D(pool_size=(2,2), strides = (2,2), name= "P_maxpool_5")(reluImg_5)

zeroPad_6 = ZeroPadding2D((1,1), name= "P_zero_6")(poolImg_5)
convImg_6  = Convolution2D(512, (3, 3), strides=(1,1), name= "P_conv_6")(zeroPad_6)
batchImg_6 = BatchNormalization(epsilon=1.0000e-04, name= "P_batch_6")(convImg_6)
reluImg_6  = ReLU(name= "P_relu_6")(batchImg_6)

# upsample
de_deconv_1 = Conv2DTranspose(512, (4, 4), strides=(2,2), dilation_rate=(1,1), name= "P_de_deconv_7")(reluImg_6)
de_crop_1 = Cropping2D(cropping=((1, 1), (1, 1)))(de_deconv_1)
de_zeroPad_1 = ZeroPadding2D((1,1), name= "P_de_zero_7")(de_crop_1)
de_convImg_1 = Convolution2D(256, (3, 3), strides=(1,1), name= "P_de_conv_7")(de_zeroPad_1)
de_batchImg_1 = BatchNormalization(epsilon=1.0000e-04, name= "P_de_batch_7")(de_convImg_1)
## bw'lere hiç bakmadan ekledik. ne zaman halledicez?
de_reluImg_1 = ReLU(name= "P_de_relu_7")(de_batchImg_1)
de_sumImg_1 = Add(name= "add_1")([de_reluImg_1, reluImg_5])


de_deconv_2 = Conv2DTranspose(256, (4, 4), strides=(2,2), dilation_rate=(1,1), name= "P_de_deconv_8")(de_sumImg_1)
de_crop_2 = Cropping2D(cropping=((1, 1), (1, 1)))(de_deconv_2)
de_zeroPad_2 = ZeroPadding2D((1,1), name= "P_de_zero_8")(de_crop_2)
de_convImg_2 = Convolution2D(128, (3, 3), strides=(1,1), name= "P_de_conv_8")(de_zeroPad_2)
de_batchImg_2 = BatchNormalization(epsilon=1.0000e-04, name= "P_de_batch_8")(de_convImg_2)
de_reluImg_2 = ReLU(name= "P_de_relu_8")(de_batchImg_2)
de_sumImg_2 = Add(name = "add_2")([de_reluImg_2, reluImg_4])

de_deconv_3 = Conv2DTranspose(128, (4, 4), strides=(2,2), dilation_rate=(1,1), name= "P_de_deconv_9")(de_sumImg_2)
de_crop_3 = Cropping2D(cropping=((1, 1), (1, 1)))(de_deconv_3)
de_zeroPad_3 = ZeroPadding2D((1,1), name= "P_de_zero_9")(de_crop_3)
de_convImg_3 = Convolution2D(64, (3, 3), strides=(1,1), name= "P_de_conv_9")(de_zeroPad_3)
de_batchImg_3 = BatchNormalization(epsilon=1.0000e-04, name= "P_de_batch_9")(de_convImg_3)
de_reluImg_3 = ReLU(name= "P_de_relu_9")(de_batchImg_3)
de_sumImg_3 = Add(name = "add_3")([de_reluImg_3, reluImg_3])

de_deconv_4 = Conv2DTranspose(64, (4, 4), strides=(2,2), dilation_rate=(1,1), name= "P_de_deconv_10")(de_sumImg_3)
de_crop_4 = Cropping2D(cropping=((1, 1), (1, 1)))(de_deconv_4)
de_zeroPad_4 = ZeroPadding2D((1,1), name= "P_de_zero_10")(de_crop_4)
de_convImg_4 = Convolution2D(32, (3, 3), strides=(1,1), name= "P_de_conv_10")(de_zeroPad_4)
de_batchImg_4 = BatchNormalization(epsilon=1.0000e-04, name= "P_de_batch_10")(de_convImg_4)
de_reluImg_4 = ReLU(name= "P_de_relu_10")(de_batchImg_4)
de_sumImg_4 = Add(name = "add_4")([de_reluImg_4, reluImg_2])

de_deconv_5 = Conv2DTranspose(32, (4, 4), strides=(2,2), dilation_rate=(1,1), name= "P_de_deconv_11")(de_sumImg_4)
de_crop_5 = Cropping2D(cropping=((1, 1), (1, 1)))(de_deconv_5)
de_zeroPad_5 = ZeroPadding2D((1,1), name= "P_de_zero_11")(de_crop_5)
de_convImg_5 = Convolution2D(16, (3, 3), strides=(1,1), name= "P_de_conv_11")(de_zeroPad_5)
de_batchImg_5 = BatchNormalization(epsilon=1.0000e-04, name= "P_de_batch_11")(de_convImg_5)
de_reluImg_5 = ReLU(name= "P_de_relu_11")(de_batchImg_5)

de_zeroPad_6 = ZeroPadding2D((1,1), name= "P_de_zero_12")(de_reluImg_5)
de_convImg_parsing = Convolution2D(11, (3, 3), strides=(1,1), name= "P_de_conv_12")(de_zeroPad_6)


## Generative Network

In [ ]:

modelG_Input = Input(shape=(128,128,11))
G_softImg_1 = Softmax(name= "G_soft_1")(modelG_Input)
G_poolImg_1 = MaxPooling2D(pool_size=(2,2), strides = (2,2), name= "G_maxpool_1")(G_softImg_1)
G_concatImg_1 = Concatenate(name= "G_concat_1")([inputImg0, G_poolImg_1])


G_zeroPad_1 = ZeroPadding2D((5,5), name= "G_zero_1")(G_concatImg_1)
G_convImg_1 = Convolution2D(64, (11, 11), strides=(1,1), name= "G_conv_1")(G_zeroPad_1)
G_reluImg_1 = ReLU(name= "G_relu_1")(G_convImg_1)

G_zeroPad_2 = ZeroPadding2D((2,2), name= "G_zero_2")(G_reluImg_1)
G_convImg_2 = Convolution2D(64, (5, 5), strides=(1,1), name= "G_conv_2")(G_zeroPad_2)
G_reluImg_2 = ReLU(name= "G_relu_2")(G_convImg_2)

G_zeroPad_3 = ZeroPadding2D((2,2), name= "G_zero_3")(G_reluImg_2)
G_convImg_3 = Convolution2D(64, (5, 5), strides=(1,1), name= "G_conv_3")(G_zeroPad_3)
G_reluImg_3 = ReLU(name= "G_relu_3")(G_convImg_3)

G_res1_zeroPad_1_1 = ZeroPadding2D((2,2), name= "G_res1_zero_1_1")(G_reluImg_3)
G_res1_convImg_1_1 = Convolution2D(64, (5, 5), strides=(1,1), name= "G_res1_conv_1_1")(G_res1_zeroPad_1_1)
G_res1_reluImg_1_1 = ReLU(name= "G_res1_relu_1_1")(G_res1_convImg_1_1)
G_res1_zeroPad_1_2 = ZeroPadding2D((2,2), name= "G_res1_zero_1_2")(G_res1_reluImg_1_1)
G_res1_convImg_1_2 = Convolution2D(64, (5, 5), strides=(1,1), name= "G_res1_conv_1_2")(G_res1_zeroPad_1_2) #### !!!!!!!!!!!11
G_res1_sumImg_1 = Add(name = "add_5")([G_res1_convImg_1_2, G_reluImg_3])
G_res1_reluImg_1_2 = ReLU(name= "G_res1_relu_1_2")(G_res1_sumImg_1)

G_res1_zeroPad_2_1 = ZeroPadding2D((2,2), name= "G_res1_zero_2_1")(G_res1_reluImg_1_2)
G_res1_convImg_2_1 = Convolution2D(64, (5, 5), strides=(1,1), name= "G_res1_conv_2_1")(G_res1_zeroPad_2_1)
G_res1_reluImg_2_1 = ReLU(name= "G_res1_relu_2_1")(G_res1_convImg_2_1)
G_res1_zeroPad_2_2 = ZeroPadding2D((2,2), name= "G_res1_zero_2_2")(G_res1_reluImg_2_1)
G_res1_convImg_2_2 = Convolution2D(64, (5, 5), strides=(1,1), name= "G_res1_conv_2_2")(G_res1_zeroPad_2_2)
G_res1_sumImg_2 = Add(name ="add_6")([G_res1_convImg_2_2, G_res1_sumImg_1])
G_res1_reluImg_2_2 = ReLU(name= "G_res1_relu_2_2")(G_res1_sumImg_2)

G_res1_zeroPad_3_1 = ZeroPadding2D((2,2), name= "G_res1_zero_3_1")(G_res1_reluImg_2_2)
G_res1_convImg_3_1 = Convolution2D(64, (5, 5), strides=(1,1), name= "G_res1_conv_3_1")(G_res1_zeroPad_3_1)
G_res1_reluImg_3_1 = ReLU(name= "G_res1_relu_3_1")(G_res1_convImg_3_1)
G_res1_zeroPad_3_2 = ZeroPadding2D((2,2), name= "G_res1_zero_3_2")(G_res1_reluImg_3_1)
G_res1_convImg_3_2 = Convolution2D(64, (5, 5), strides=(1,1), name= "G_res1_conv_3_2")(G_res1_zeroPad_3_2)
G_res1_sumImg_3 = Add(name = "add_7")([G_res1_convImg_3_2, G_res1_sumImg_2])
G_res1_reluImg_3_2 = ReLU(name= "G_res1_relu_3_2")(G_res1_sumImg_3)

G_res1_zeroPad_4_1 = ZeroPadding2D((2,2), name= "G_res1_zero_4_1")(G_res1_reluImg_3_2)
G_res1_convImg_4_1 = Convolution2D(64, (5, 5), strides=(1,1), name= "G_res1_conv_4_1")(G_res1_zeroPad_4_1)
G_res1_reluImg_4_1 = ReLU(name= "G_res1_relu_4_1")(G_res1_convImg_4_1)
G_res1_zeroPad_4_2 = ZeroPadding2D((2,2), name= "G_res1_zero_4_2")(G_res1_reluImg_4_1)
G_res1_convImg_4_2 = Convolution2D(64, (5, 5), strides=(1,1), name= "G_res1_conv_4_2")(G_res1_zeroPad_4_2)
G_res1_sumImg_4 = Add(name = "add_8")([G_res1_convImg_4_2, G_res1_sumImg_3])
G_res1_reluImg_4_2 = ReLU(name= "G_res1_relu_4_2")(G_res1_sumImg_4)

##### yunus emre 
G_res1_zeroPad_5_1 = ZeroPadding2D((2,2), name= "G_res1_zero_5_1")(G_res1_reluImg_4_2)
G_res1_convImg_5_1 = Convolution2D(64, (5, 5), strides=(1,1), name= "G_res1_conv_5_1")(G_res1_zeroPad_5_1)
G_res1_zeroPad_5_2 = ZeroPadding2D((2,2), name= "G_res1_zero_5_2")(G_res1_convImg_5_1)
G_res1_convImg_5_2 = Convolution2D(64, (5, 5), strides=(1,1), name= "G_res1_conv_5_2")(G_res1_zeroPad_5_2)
G_res1_sumImg_5 = Add(name = "add_9")([G_res1_convImg_5_2,G_res1_sumImg_4])
G_res1_reluImg_5_2 = ReLU(name= "G_res1_relu_5_2")(G_res1_sumImg_5)

G_res1_zeroPad_4 = ZeroPadding2D((2,2), name= "G_res1_zero_4")(G_res1_reluImg_5_2)
G_convImg_4 = Convolution2D(64, (5, 5), strides=(1,1), name= "G_res1_conv_4")(G_res1_zeroPad_4)
G_reluImg_4 = ReLU(name= "G_res1_relu_4")(G_convImg_4)

G_res1_zeroPad_5 = ZeroPadding2D((2,2), name= "G_res1_zero_5")(G_reluImg_4)
G_convImg_5 = Convolution2D(64, (5, 5), strides=(1,1), name= "G_res1_conv_5")(G_res1_zeroPad_5)
G_reluImg_5 = ReLU(name= "G_res1_relu_5")(G_convImg_5)

G_res1_zeroPad_6 = ZeroPadding2D((2,2), name= "G_res1_zero_6")(G_reluImg_5)
G_convImg_6 = Convolution2D(3, (5, 5), strides=(1,1), name= "G_res1_conv_6")(G_res1_zeroPad_6)

### SCALE_2
de_deconv = Conv2DTranspose(3, (4, 4), strides=(2,2), dilation_rate=(1,1), name= "de_deconv")(G_convImg_6)
de_crop = Cropping2D(cropping=((1, 1), (1, 1)))(de_deconv)
G_concatImg_12 = Concatenate(name= "G_concat_2")([de_crop, inputImg])
G_concatImg_2 = Concatenate(name= "G_concat_3")([G_concatImg_12, G_softImg_1])

G_convImg2_1_zero_pad = ZeroPadding2D((5,5), name= "G_convImg2_1_zero_pad")(G_concatImg_2)
G_convImg2_1 = Convolution2D(64, (11, 11), strides=(1,1), name= "G_convImg2_1")(G_convImg2_1_zero_pad)
G_reluImg2_1 = ReLU(name= "G_reluImg2_1")(G_convImg2_1)

G_convImg2_2_zero_pad = ZeroPadding2D((2,2), name= "G_convImg2_2_zero_pad")(G_reluImg2_1)
G_convImg2_2 = Convolution2D(64, (5, 5), strides=(1,1), name= "G_convImg2_2")(G_convImg2_2_zero_pad)
G_reluImg2_2 = ReLU(name= "G_reluImg2_2")(G_convImg2_2)

G_convImg2_3_zero_pad = ZeroPadding2D((2,2), name= "G_convImg2_3_zero_pad")(G_reluImg2_2)
G_convImg2_3 = Convolution2D(64, (5, 5), strides=(1,1), name= "G_convImg2_3")(G_convImg2_3_zero_pad)
G_reluImg2_3 = ReLU(name= "G_reluImg2_3")(G_convImg2_3)

G_res2_zeroPad_1_1 = ZeroPadding2D((2,2), name= "G_res2_zero_1_1")(G_reluImg2_3)
G_res2_convImg_1_1 = Convolution2D(64, (5, 5), strides=(1,1), name= "G_res2_conv_1_1")(G_res2_zeroPad_1_1)
G_res2_reluImg_1_1 = ReLU(name= "G_res2_relu_1_1")(G_res2_convImg_1_1)
G_res2_zeroPad_1_2 = ZeroPadding2D((2,2), name= "G_res2_zero_1_2")(G_res2_reluImg_1_1)
G_res2_convImg_1_2 = Convolution2D(64, (5, 5), strides=(1,1), name= "G_res2_conv_1_2")(G_res2_zeroPad_1_2)
G_res2_sumImg_1 = Add(name = "add_10")([G_res2_convImg_1_2, G_reluImg2_3])
G_res2_reluImg_1_2 = ReLU(name= "G_res2_relu_1_2")(G_res2_sumImg_1)


G_res2_zeroPad_2_1 = ZeroPadding2D((2,2), name= "G_res2_zeroPad_2_1")(G_res2_reluImg_1_2)
G_res2_convImg_2_1 = Convolution2D(64, (5, 5), strides=(1,1), name= "G_res2_conv_2_1")(G_res2_zeroPad_2_1)
G_res2_reluImg_2_1 = ReLU(name= "G_res2_relu_2_1")(G_res2_convImg_2_1)
G_res2_zeroPad_2_2 = ZeroPadding2D((2,2), name= "G_res2_zero_2_2")(G_res2_reluImg_2_1)
G_res2_convImg_2_2 = Convolution2D(64, (5, 5), strides=(1,1), name= "G_res2_conv_2_2")(G_res2_zeroPad_2_2)
G_res2_sumImg_2 = Add(name = "add_11")([G_res2_convImg_2_2, G_res2_sumImg_1])
G_res2_reluImg_2_2 = ReLU(name= "G_res2_relu_2_2")(G_res2_sumImg_2)


G_res2_zeroPad_3_1 = ZeroPadding2D((2,2), name= "G_res2_zero_3_1")(G_res2_reluImg_2_2)
G_res2_convImg_3_1_deneme = Convolution2D(64, (5, 5), strides=(1,1), name= "G_res2_conv_3_1_deneme")(G_res2_zeroPad_3_1) ### !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!1
G_res2_reluImg_3_1 = ReLU(name= "G_res2_relu_3_1")(G_res2_convImg_3_1_deneme)
G_res2_zeroPad_3_2 = ZeroPadding2D((2,2), name= "G_res2_zero_3_2")(G_res2_reluImg_3_1)
G_res2_convImg_3_2 = Convolution2D(64, (5, 5), strides=(1,1), name= "G_res2_conv_3_2")(G_res2_zeroPad_3_2)
G_res2_sumImg_3 = Add(name = "add_12")([G_res2_convImg_3_2, G_res2_sumImg_2])
G_res2_reluImg_3_2 = ReLU(name= "G_res2_relu_3_2")(G_res2_sumImg_3)


G_res2_zeroPad_4_1 = ZeroPadding2D((2,2), name= "G_res2_zero_4_1")(G_res2_reluImg_3_2)
G_res2_convImg_4_1 = Convolution2D(64, (5, 5), strides=(1,1), name= "G_res2_conv_4_1")(G_res2_zeroPad_4_1)
G_res2_reluImg_4_1 = ReLU(name= "G_res2_relu_4_1")(G_res2_convImg_4_1)
G_res2_zeroPad_4_2 = ZeroPadding2D((2,2), name= "G_res2_zero_4_2")(G_res2_reluImg_4_1)
G_res2_convImg_4_2 = Convolution2D(64, (5, 5), strides=(1,1), name= "G_res2_conv_4_2")(G_res2_zeroPad_4_2)
G_res2_sumImg_4 = Add(name = "add_13")([G_res2_convImg_4_2, G_res2_sumImg_3])
G_res2_reluImg_4_2 = ReLU(name= "G_res2_relu_4_2")(G_res2_sumImg_4)


G_res2_zeroPad_5_1 = ZeroPadding2D((2,2), name= "G_res2_zero_5_1")(G_res2_reluImg_4_2)
G_res2_convImg_5_1 = Convolution2D(64, (5, 5), strides=(1,1), name= "G_res2_conv_5_1")(G_res2_zeroPad_5_1)
G_res2_reluImg_5_1 = ReLU(name= "G_res2_relu_5_1")(G_res2_convImg_5_1)
G_res2_zeroPad_5_2 = ZeroPadding2D((2,2), name= "G_res2_zero_5_2")(G_res2_reluImg_5_1)
G_res2_convImg_5_2 = Convolution2D(64, (5, 5), strides=(1,1), name= "G_res2_conv_5_2")(G_res2_zeroPad_5_2)
G_res2_sumImg_5 = Add(name = "add_14")([G_res2_convImg_5_2, G_res2_sumImg_4])
G_res2_reluImg_5_2 = ReLU(name= "G_res2_relu_5_2")(G_res2_sumImg_5)


G_2convImg_4_zero_pad = ZeroPadding2D((2,2), name= "G_2convImg_4_zero_pad")(G_res2_reluImg_5_2)
G_2convImg_4 = Convolution2D(64, (5, 5), strides=(1,1), name= "G_convImg4")(G_2convImg_4_zero_pad)
G_2reluImg_4 = ReLU(name= "G_reluImg4")(G_2convImg_4)

G_2convImg_5_zero_pad = ZeroPadding2D((2,2), name= "G_2convImg_5_zero_pad")(G_2reluImg_4)
G_2convImg_5 = Convolution2D(64, (5, 5), strides=(1,1), name= "G_convImg5")(G_2convImg_5_zero_pad)
G_2reluImg_5 = ReLU(name= "G_reluImg5")(G_2convImg_5)


G_convImg_6_zero_pad = ZeroPadding2D((2,2), name= "G_convImg_6_zero_pad")(G_2reluImg_5)
G_convImg_6_last = Convolution2D(3, (5, 5), strides=(1,1), name= "G_convImg_6_last")(G_convImg_6_zero_pad)

modelP = Model(inputs=[inputImg], outputs=de_convImg_parsing)
modelG = Model(inputs=[modelG_Input,inputImg0, inputImg ], outputs=G_convImg_6_last)

In [ ]:
modelG.summary()

In [ ]:
modelP.layers[3].weights
print(netP[0,4].bm.shape)

### Getting the indices of the layers to set the model weights and setting them

In [ ]:
conv_counter = 1
norm_counter = 1
tr_conv_counter = 1
b_conv_list = []
b_batch_list = []
b_tconv_list = []

for i in range(len(modelP.layers)):
    layer_name = str(modelP.layers[i])


    if "BatchNormalization" in layer_name:
        norm_counter += 1
        b_batch_list.append(i)
        
    if "Conv2DTranspose" in layer_name:
        tr_conv_counter += 1
        b_tconv_list.append(i)
    
    elif "Conv2D" in layer_name:
        b_conv_list.append(i)
        conv_counter += 1
        #print(modelP.layers[i].name)
conv_counter
# "normalization"
# convolution

In [ ]:
counter = 0
model_index = 0
tconv_index = 0
batch_index = 0

for i in range(netP.shape[1]):
    field_names = netP[0][i].__dict__["_fieldnames"]
    a = i + 1
        
    for name in field_names:
        #if model_index != 28:
        if name == "w":
            print(i, model_index, modelP.layers[b_conv_list[model_index]].weights[0].shape,"-------", netP[0][i].w.shape)
            modelP.layers[b_conv_list[model_index]].set_weights([netP[0][i].w, netP[0][i].b.reshape(netP[0][i].b.shape[0], )])
            model_index += 1
            counter +=1

        
        if name == "bw":
            print(i)
            if batch_index == len(b_batch_list):
                print("sona gelindi")
                break
            print(i, batch_index, modelP.layers[b_batch_list[batch_index]].weights[0].shape,"bbbbbbbbb", netP[0][i].bw.shape)
            bm1 = np.copy(netP[0][i].bm[:,0].reshape(netP[0][i].bb.shape[0],))
            bm2 = np.copy(netP[0][i].bm[:,1].reshape(netP[0][i].bb.shape[0],))

            modelP.layers[b_batch_list[batch_index]].set_weights([ netP[0][i].bw.reshape(netP[0,i].bw.shape[0],), netP[0][i].bb.reshape(netP[0][i].bb.shape[0]), bm1, bm2])
            #print(i, batch_index, modelP.layers[batch_list[batch_index]].weights[0].shape,"-------", netP[0][i].w.shape)
            batch_index += 1
            counter +=1

        if name == "upw":
            if netP[0][i].upb.shape[0] != 1:
                print(str(a), name + " shape", net[0][i].upw.shape )
                modelP.layers[b_tconv_list[tconv_index]].set_weights([netP[0][i].upw, netP[0][i].upb.reshape(netP[0][i].upb.shape[0])])
                
                tconv_index += 1
                counter +=1


In [ ]:
## Finding the Conv2dTranspose and Con2d layers in the model so that we can set their weights easily.
conv_counter = 1
norm_counter = 1
tr_conv_counter = 1
conv_list = []
batch_list = []
tconv_list = []

for i in range(len(modelG.layers)):
    layer_name = str(modelG.layers[i])
    
    if "Conv2D" in layer_name or "Conv2DTranspose" in layer_name:
        conv_list.append(i)
        conv_counter += 1

In [ ]:
counter = 0
model_index = 0
tconv_index = 0
batch_index = 0

for i in range(netG.shape[1]):
    field_names = netG[0][i].__dict__["_fieldnames"]
    a = i + 1
    for name in field_names:
        try: 

            #if model_index != 28:
            if name == "w":
                #print("shapeeeeee", np.array(modelG.layers[conv_list[model_index]].get_weights()).shape)
                print("conv",i, model_index, modelG.layers[conv_list[model_index]].weights[0].shape,"-------", netG[0][i].w.shape)

                modelG.layers[conv_list[model_index]].set_weights([netG[0][i].w, netG[0][i].b.reshape(netG[0][i].b.shape[0], )])
                if np.all(modelG.layers[conv_list[model_index]].get_weights()[1] == netG[0][i].b.reshape(netG[0][i].b.shape[0], )): 
                    pass
                else:
                    print("there is an error while setting the weights of the layer")
                    break 
                model_index += 1
                counter +=1

            
            if name == "bw":
                print(i)
                if batch_index == len(batch_list):
                    print("sona gelindi")
                    break
                print("batch", i, batch_index, modelG.layers[batch_list[batch_index]].weights[0].shape,"bbbbbbbbb", netG[0][i].bw.shape)
                bm1 = np.copy(netG[0][i].bm[:,0].reshape(netG[0][i].bb.shape[0],))
                bm2 = np.copy(netG[0][i].bm[:,1].reshape(netG[0][i].bb.shape[0],))

                modelG.layers[batch_list[batch_index]].set_weights([ netG[0][i].bw.reshape(netG[0,i].bw.shape[0],), netG[0][i].bb.reshape(netG[0][i].bb.shape[0]), bm1, bm2])
                #print(i, batch_index, modelG.layers[batch_list[batch_index]].weights[0].shape,"-------", netG[0][i].w.shape)
                batch_index += 1
                counter +=1

            if name == "upw":
                if netG[0][i].upb.shape[0] != 1:
                    print("transpose",str(a), name + " shape", netG[0][i].upw.shape )
                    modelG.layers[tconv_list[tconv_index]].set_weights([netG[0][i].upw, netG[0][i].upb.reshape(netG[0][i].upb.shape[0])])
                    
                    tconv_index += 1
                    counter +=1
        except Exception as e:
            print("------------------------------", e)
            model_index += 1

## Running the Model
Try the model performance with an example image given by the authors of the paper

In [ ]:
import cv2 
image = cv2.imread("example.png")
#image = cv2.resize(image, (128,128), interpolation=cv2.INTER_CUBIC)
#cv2.imwrite("small.png", image)
image = cv2.cvtColor(image ,cv2.COLOR_BGR2RGB)
image=image.astype(np.float64)/255.0
image_resized = np.reshape(image, (1,128,128,3)) 
print(image_resized.shape)
print(image.shape)
#save model and structure alone
#modelParsing.save("model.hdf5")
#modelParsing.compile(optimizer='adam')
result = modelP.predict(image_resized)

img_new = cv2.resize(image, (64,64))
img_small = np.reshape(img_new, (1,64,64,3))
res = modelG.predict([result, img_small, image_resized] )

#for i in range(result.shape[2]):
#    cv2.imwrite("semantic"+ str(i) +".png", result[:,:,i])

res = res.reshape((128,128,3))

res = np.clip(res, 0.0, 1.0) * 255.0
res = res.astype(np.uint8)
res = cv2.cvtColor(res ,cv2.COLOR_RGB2BGR)
#res_rgb = cv2.cvtColor(res, cv2.COLOR_BGR2RGB)
cv2.imwrite("res2.png", res)